In [1]:
import pandas as pd
import io
import requests
wine="white"#red/white
s=requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-"+wine+".csv"
               ).content
data=pd.read_csv(io.StringIO(s.decode('utf-8')),error_bad_lines=False,sep=';')
X, y=data.iloc[:,:-1].values,data.iloc[:,-1].values

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)

In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=9)#Covering ~ 95% variance
X=pca.fit_transform(X)
print(pca.explained_variance_ratio_)
print('Total variance covered: ',sum(pca.explained_variance_ratio_))

[ 0.29293217  0.14320363  0.11106103  0.09259294  0.08848496  0.08534014
  0.06605437  0.05448713  0.03764942]
Total variance covered:  0.971805797788


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,shuffle=False)

In [16]:
from sklearn import ensemble
import time
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
start=time.time()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
end=time.time()
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
print("MSE: %.4f" % mse)
print('Train + test time: ',end-start,'sec')

MSE: 0.4646
Train + test time:  3.9342827796936035 sec


In [17]:
import tensorflow as tf
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          activation_fn = tf.nn.relu, hidden_units=[80, 40,20])
regressor = tf.contrib.learn.SKCompat(regressor) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028EAC5A3978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\vikhy\\AppData\\Local\\Temp\\tmpn9zqwxfs'}


In [18]:
start=time.time()
regressor.fit(X_train, y_train, batch_size=10, steps=2000)
y_pred = regressor.predict(X_test)
end=time.time()

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\vikhy\AppData\Local\Temp\tmpn9zqwxfs\model.ckpt.
INFO:tensorflow:loss = 34.572, step = 1
INFO:tensorflow:global_step/sec: 610.236
INFO:tensorflow:loss = 1.51908, step = 101 (0.164 sec)
INFO:tensorflow:global_step/sec: 575.735
INFO:tensorflow:loss = 0.152114, step = 201 (0.174 sec)
INFO:tensorflow:global_step/sec: 473.631
INFO:tensorflow:loss = 0.693285, step = 301 (0.228 sec)
INFO:tensorflow:global_step/sec: 542.969
INFO:tensorflow:loss = 0.171251, step = 401 (0.184 sec)
INFO:tensorflow:global_step/sec: 519.727
INFO:tensorflow:loss = 0.39261, step = 501 (0.176 sec)


In [19]:
print(mean_squared_error(y_test, y_pred['scores']))
print('Train + test time: ',end-start,'sec')

0.420948384996
Train + test time:  10.755764722824097 sec
